<a href="https://colab.research.google.com/github/raedfesesi/colab_1/blob/main/Finetune_Mistral7B_on_a_single_GPU_with_PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Mistral-7b AI on a Single GPU using PEFT LORA with Google Colab
Welcome to this notebook that will show you how to finetune Mistral-7b using the  recent peft library and bitsandbytes for loading large models in 4-bit.

The fine-tuning method will rely on a recent method called "Low Rank Adapters" (LoRA), instead of fine-tuning the entire model you just have to fine-tune these adapters and load them properly inside the model. After fine-tuning the model you can also share your adapters on the 🤗 Hub and load them very easily. Let's get started!

Note that this could be used for any model that supports device_map (i.e. loading the model with accelerate).

## Step 0 -  Define some helper functions :
1. Enable text wrapping so we don't have to scrool horizontally
2. Define a wrapper function which pass our query to the model for inference and return decoded model's completion(response).


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


Let's define a wrapper function which will get completion from the model from a user question

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  {query}

  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

## Step 1 - Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [ ]:
model_id = "mistralai/Mistral-7B-v0.1"


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Run a inference on the base model. The model does not seem to understand our instruction and gives us a list of questions related to our query.

In [ ]:
result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Will capital gains affect my tax bracket?

  ### Answer:
  </s>ю


  ### Question:
  Would paying taxes earlier this year result in getting money back at tax time in 2021?

  ### Answer:
  Ї


  ### Question:
  Should we pay taxes as a corporation or as a partnership to maximize our savings at tax time?

  ### Answer

  Щ


  ### Question:
  What are different tax rates in different states?

  ### Answer:
  д


  ### Question:
  What are the biggest impactful areas to consider when looking for savings when preparing your taxes?

  ### Response

  Є


  ### Question:
  Given the new tax changes for 2021, what should we consider to be the most important changes?

  ### Answer:
  Ґ


  ### Question:
  How should we plan for taxes in advance of changes to tax rates?

  ### Answer:

  А</s>


## Step 3 - Load dataset for finetuning

Let's load a dataset on finance, to fine tune our model on basic finance knowledges. In this guide, we'll load 10% data from the original dataset for the sake of the demo just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
from datasets import load_dataset

data = load_dataset("gbharti/finance-alpaca", split='train[:10%]')

# Explore the data
df = data.to_pandas()
df.head(10)




Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

,instruction,text,input,output
0,"For a car, what scams can be plotted with 0% f...",,,The car deal makes money 3 ways. If you pay in...
1,Why does it matter if a Central Bank has a neg...,,,"That is kind of the point, one of the hopes is..."
2,Where should I be investing my money?,,,"Pay off your debt. As you witnessed, no ""inve..."
3,Specifically when do options expire?,,,"Equity options, at least those traded in the A..."
4,Negative Balance from Automatic Options Exerci...,,,"Automatic exercisions can be extremely risky, ..."
5,Approximation of equity value for company in d...,,,"Generally ""default"" means that the company can..."
6,Is it true that 90% of investors lose their mo...,,,The game is not zero sum. When a friend and I ...
7,Can a company charge you for services never re...,,,"In general, you can only be charged for servic..."
8,Working out if I should be registered as self-...,,,Being self employed just means you fill out so...
9,About eToro investments,,,"For eToro, just like any other brokerage firm,..."


Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    # Samples with additional context into.
    if data_point['input']:
        text = 'Below is an instruction that describes a task, paired with an input that provides' \
               ' further context. Write a response that appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Input:\n{data_point["input"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'

    # Without
    else:
        text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in data]
data = data.add_column("prompt", text_column)

We'll need to tokenize our data so the model can understand.


In [ ]:
data = data.shuffle(seed=1234)  # Shuffle dataset here
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/6891 [00:00<?, ? examples/s]

Split dataset into 90% for training and 10% for testing

In [ ]:
data = data.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)
print_trainable_parameters(peft_model)

trainable params: 6815744 || all params: 3765702656 || trainable%: 0.18099527824217038


Add adapter to the Model

In [ ]:
# model.add_adapter(lora_config, adapter_name="adapter")

## Step 5 - Run the training!

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
data = load_dataset("ronal999/finance-alpaca", split='train[:10%]')
data = data.shuffle(seed=1234)  # Shuffle dataset here
data = data.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj","o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token



trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs_mistral_b_finance_finetuned_test",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        push_to_hub=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


Start the training

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()




You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.133900
2,2.763300
3,2.500300
4,2.187000
5,2.104900
6,2.126700
7,2.040800
8,2.046800
9,1.591500
10,2.107600


/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


TrainOutput(global_step=100, training_loss=1.9380053246021272, metrics={'train_runtime': 292.558, 'train_samples_per_second': 1.367, 'train_steps_per_second': 0.342, 'total_flos': 5102303643795456.0, 'train_loss': 1.9380053246021272, 'epoch': 0.06})

 Share adapters on the 🤗 Hub

In [ ]:
model.push_to_hub("mistral_b_finance_finetuned_test")
tokenizer.push_to_hub("mistral_b_finance_finetuned_test")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Ronal999/mistral_b_finance_finetuned_test/commit/043fc3e3b8bdd04e69f164d4e86d2b18dd8429f4', commit_message='Upload tokenizer', commit_description='', oid='043fc3e3b8bdd04e69f164d4e86d2b18dd8429f4', pr_url=None, pr_revision=None, pr_num=None)

## Step 6 Evaluating the model qualitatively: run an inference!



In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Load directly adapters from the Hub using the command below

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Ronal999/mistral_b_finance_finetuned_test"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


You can then directly use the trained model that you have loaded from the 🤗 Hub for inference as you would do it usually in transformers.

In [ ]:
result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
print(result)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Will capital gains affect my tax bracket?

  ### Answer:
   Long term and short term capital gains and losses are taxed at different tax brackets. If you have long term gains when calculating ordinary taxes, they are taxed as ordinary income at a lower rate than at the long term capital gains rate. It also helps to remember that it's the change in net worth that drives long and short term taxes. If I buy a stock for $4 per share and sell it for $16 per share, even if the 4:16 ratio is 50% more than your overall cost basis, you have a 4 to 1 capital gain for tax purposes. If you owned the stock for less than a year the gain is short term, taxes as ordinary income. So long term capital gains can be as low as 0%. The tax rate on your capital gains can easily be calculated using this table. It's important to note that the tax code assumes all capital gains are 